In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-2.4.0/spark-2.4.0-bin-hadoop2.7.tgz
!tar -xvf spark-2.4.0-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.0-bin-hadoop2.7"
import findspark
findspark.init()

spark-2.4.0-bin-hadoop2.7/
spark-2.4.0-bin-hadoop2.7/python/
spark-2.4.0-bin-hadoop2.7/python/setup.cfg
spark-2.4.0-bin-hadoop2.7/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/resultiterable.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/
spark-2.4.0-bin-hadoop2.7/python/pyspark/python/pyspark/shell.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/heapq3.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/join.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/version.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/rdd.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/java_gateway.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/find_spark_home.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/_globals.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/worker.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/accumulators.py
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/
spark-2.4.0-bin-hadoop2.7/python/pyspark/mllib/feature.py
spark-2.4.0-bin-hadoop2.7/python/pyspark

In [2]:
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)

Mounted at /content/gdrive


In [3]:
%cd '/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/LDS9_K273_BuiThiHuong_Bai_tap_cong_diem/'

/content/gdrive/My Drive/LDS9_K273_ONLINE_BuiThiHuong/LDS9_K273_BuiThiHuong_Bai_tap_cong_diem


In [4]:
# import findspark
# findspark.init()

In [5]:
# import libraries
from pyspark import SparkContext
from pyspark.conf import SparkConf
from pyspark.sql import SparkSession
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from datetime import datetime
import time
from pyspark.sql.functions import mean, stddev, col, log
from pyspark.sql.functions import to_date, dayofweek, to_timestamp
from pyspark.sql.functions import *
from pyspark.sql import types 
from pyspark.sql.functions import col, udf
from pyspark.sql.types import DateType
from pyspark.sql.functions import year, month
from pyspark.sql.functions import dayofmonth, weekofyear
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import coalesce, first, lit
from pyspark.ml.feature import Binarizer
from pyspark.ml.feature import Bucketizer
from pyspark.ml.feature import OneHotEncoder, StringIndexer
from pyspark.sql.functions import regexp_extract, col
from pyspark.sql.functions import datediff
from pyspark.sql.functions import when

from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.classification import LogisticRegressionModel
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

from pyspark.ml import Pipeline, PipelineModel

In [6]:
sc =SparkContext()

In [7]:
spark = SparkSession(sc)

## Read data

In [8]:
start_time = time.time()

In [9]:
# Use Spark to read in the Ecommerce Customers csv file.
data = spark.read.csv("customer_churn.csv", inferSchema=True,header=True)

In [10]:
# Print the Schema of the DataFrame
data.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



In [11]:
print((data.count(), len(data.columns)))

(900, 10)


In [12]:
data.head(1)

[Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)]

In [13]:
for item in data.head():
    print(item)

Cameron Williams
42.0
11066.8
0
7.22
8.0
2013-08-30 07:00:40
10265 Elizabeth Mission Barkerburgh, AK 89518
Harvey LLC
1


In [14]:
data.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

In [15]:
data.show(5)

+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|           Names| Age|Total_Purchase|Account_Manager|Years|Num_Sites|       Onboard_date|            Location|             Company|Churn|
+----------------+----+--------------+---------------+-----+---------+-------------------+--------------------+--------------------+-----+
|Cameron Williams|42.0|       11066.8|              0| 7.22|      8.0|2013-08-30 07:00:40|10265 Elizabeth M...|          Harvey LLC|    1|
|   Kevin Mueller|41.0|      11916.22|              0|  6.5|     11.0|2013-08-13 00:38:46|6157 Frank Garden...|          Wilson PLC|    1|
|     Eric Lozano|38.0|      12884.75|              0| 6.67|     12.0|2016-06-29 06:20:07|1331 Keith Court ...|Miller, Johnson a...|    1|
|   Phillip White|42.0|       8010.76|              0| 6.71|     10.0|2014-04-22 12:43:12|13120 Daniel Moun...|           Smith Inc|    1|
|  Cynthia Norton|37.0|    

In [16]:
data_pre = data.select(['Age',
                          'Total_Purchase',
                          'Years',
                          'Num_Sites',
                          'Onboard_date',
                          'Churn'
                          ]).na.drop()

In [17]:
data_pre = data_pre.withColumn('label', data_pre.Churn)

In [18]:
data_pre.show(5)

+----+--------------+-----+---------+-------------------+-----+-----+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|Churn|label|
+----+--------------+-----+---------+-------------------+-----+-----+
|42.0|       11066.8| 7.22|      8.0|2013-08-30 07:00:40|    1|    1|
|41.0|      11916.22|  6.5|     11.0|2013-08-13 00:38:46|    1|    1|
|38.0|      12884.75| 6.67|     12.0|2016-06-29 06:20:07|    1|    1|
|42.0|       8010.76| 6.71|     10.0|2014-04-22 12:43:12|    1|    1|
|37.0|       9191.58| 5.56|      9.0|2016-01-19 15:31:15|    1|    1|
+----+--------------+-----+---------+-------------------+-----+-----+
only showing top 5 rows



## EDA

### Continious variavles

In [19]:
data_pre.columns

['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Churn',
 'label']

In [20]:
lst_cont = ['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 ]

data.select(lst_cont).describe().show()

+-------+-----------------+-----------------+-----------------+------------------+
|summary|              Age|   Total_Purchase|            Years|         Num_Sites|
+-------+-----------------+-----------------+-----------------+------------------+
|  count|              900|              900|              900|               900|
|   mean|41.81666666666667|10062.82403333334| 5.27315555555555| 8.587777777777777|
| stddev|6.127560416916251|2408.644531858096|1.274449013194616|1.7648355920350969|
|    min|             22.0|            100.0|              1.0|               3.0|
|    max|             65.0|         18026.01|             9.15|              14.0|
+-------+-----------------+-----------------+-----------------+------------------+



### Category variables

In [21]:
lst_cate = ['Churn']
for i in lst_cate:
  data.groupBy(i).count().show()

+-----+-----+
|Churn|count|
+-----+-----+
|    1|  150|
|    0|  750|
+-----+-----+



## Vector Assembler

In [22]:
data_pre.columns

['Age',
 'Total_Purchase',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Churn',
 'label']

In [23]:
# Create an assembler
assembler = VectorAssembler(inputCols=['Age', 
                                       'Total_Purchase', 
                                       'Years', 
                                       'Num_Sites'                                        
                                       ],
                            outputCol='features'
                            ) # inputs

In [24]:
data_pre = assembler.transform(data_pre)

In [25]:
data_pre.show(5)

+----+--------------+-----+---------+-------------------+-----+-----+--------------------+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|Churn|label|            features|
+----+--------------+-----+---------+-------------------+-----+-----+--------------------+
|42.0|       11066.8| 7.22|      8.0|2013-08-30 07:00:40|    1|    1|[42.0,11066.8,7.2...|
|41.0|      11916.22|  6.5|     11.0|2013-08-13 00:38:46|    1|    1|[41.0,11916.22,6....|
|38.0|      12884.75| 6.67|     12.0|2016-06-29 06:20:07|    1|    1|[38.0,12884.75,6....|
|42.0|       8010.76| 6.71|     10.0|2014-04-22 12:43:12|    1|    1|[42.0,8010.76,6.7...|
|37.0|       9191.58| 5.56|      9.0|2016-01-19 15:31:15|    1|    1|[37.0,9191.58,5.5...|
+----+--------------+-----+---------+-------------------+-----+-----+--------------------+
only showing top 5 rows



## Train Test splitting

In [26]:
final_data = data_pre.select('features','label')

In [27]:
# Train, Test split with ratio: 70:30
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [28]:
train_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                630|
|   mean|0.16666666666666666|
| stddev|0.37297412501838156|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



In [29]:
test_data.describe().show()

+-------+-------------------+
|summary|              label|
+-------+-------------------+
|  count|                270|
|   mean|0.16666666666666666|
| stddev| 0.3733700636914621|
|    min|                  0|
|    max|                  1|
+-------+-------------------+



## Build model

In [30]:
# Create a Logistic Regression Model object
lg = LogisticRegression(featuresCol='features',
                      labelCol='label',
                      predictionCol='prediction')
# => default: featuresCol='features', labelCol='label', predictionCol='prediction'

In [31]:
lgModel = lg.fit(train_data)

## Model evaluation

In [32]:
# check test dataset
test_prediction = lgModel.transform(test_data)

In [33]:
# Inspect results
test_prediction.select('label', 'prediction').show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 10 rows



In [34]:
# create predictions for the testing data and show confusion matrix
test_prediction.groupBy('label', 'prediction').count().orderBy('label').show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       0.0|  220|
|    0|       1.0|    5|
|    1|       0.0|   21|
|    1|       1.0|   24|
+-----+----------+-----+



In [35]:
# calculte the elements of the confusion matrix
TN = test_prediction.filter('prediction = 0 AND label = prediction').count()
TP = test_prediction.filter('prediction = 1 AND label = prediction').count()
FN = test_prediction.filter('prediction = 0 AND label != prediction').count()
FP = test_prediction.filter('prediction = 1 AND label != prediction').count()

In [36]:
# calculate precision and recall
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('precision = {:.2f}\nrecall = {:.2f}'.format(precision, recall))

precision = 0.83
recall = 0.53


In [37]:
acc = (TP+TN)/(TP+TN+FP+FN)
acc

0.9037037037037037

In [38]:
# find weighted precision
multi_evaluator = MulticlassClassificationEvaluator()
acc = multi_evaluator.evaluate(test_prediction, {multi_evaluator.metricName:'accuracy'})

In [39]:
acc 

0.9037037037037037

In [40]:
test_evaluation = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                             labelCol='label')

In [41]:
AUC = test_evaluation.evaluate(test_prediction)
AUC

0.7555555555555554

## Save and Load model

In [42]:
# Save model
lgModel.write().overwrite().save('lgModel_Customer_Churn')

In [43]:
# Load model
lgModel2 = LogisticRegressionModel.load('lgModel_Customer_Churn')

## New data prediction

In [44]:
# Predict new values (assuming select test_data)
unlabeled_data = test_data.select('features')

In [45]:
predictions = lgModel2.transform(unlabeled_data)

In [46]:
predictions.show(5)

+--------------------+--------------------+--------------------+----------+
|            features|       rawPrediction|         probability|prediction|
+--------------------+--------------------+--------------------+----------+
|[28.0,8670.98,3.9...|[6.91612432387496...|[0.99900931654890...|       0.0|
|[29.0,9617.59,5.4...|[3.72458767748666...|[0.97644516938439...|       0.0|
|[29.0,11274.46,4....|[4.40112783180121...|[0.98788507051488...|       0.0|
|[29.0,12711.15,5....|[4.85226040260038...|[0.99224983200614...|       0.0|
|[30.0,8677.28,7.3...|[3.75969731914256...|[0.97723932509232...|       0.0|
+--------------------+--------------------+--------------------+----------+
only showing top 5 rows



## Pipeline

### create models

In [47]:
# Create an assembler
assembler = VectorAssembler(inputCols=['Age', 
                                       'Total_Purchase', 
                                       'Years', 
                                       'Num_Sites'                                     
                                       ],
                            outputCol='features'
                            ) # inputs

# Logistic Regression
lg = LogisticRegression(featuresCol='features',
                      labelCol='label',
                      predictionCol='prediction')

In [48]:
# construct pipeline model
pipeline = Pipeline(stages=[assembler, lg])

train, test splitting

In [49]:
data_pre2 = data.select(['Age',
                          'Total_Purchase',
                          'Years',
                          'Num_Sites',
                          'Onboard_date',
                          'Churn'
                          ]).na.drop()

In [50]:
data_pre2 = data_pre2.withColumn('label', data_pre2.Churn)

In [51]:
train_data2, test_data2 = data_pre2.randomSplit([0.7,0.3])

In [52]:
train_data2.show(5)

+----+--------------+-----+---------+-------------------+-----+-----+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|Churn|label|
+----+--------------+-----+---------+-------------------+-----+-----+
|25.0|       9672.03| 5.49|      8.0|2006-01-02 04:16:13|    0|    0|
|26.0|       8787.39| 5.42|     11.0|2014-12-18 19:58:36|    1|    1|
|26.0|       8939.61| 4.54|      7.0|2007-09-22 09:46:47|    0|    0|
|28.0|       9090.43| 5.74|     10.0|2014-06-18 00:45:51|    0|    0|
|28.0|      11128.95| 5.12|      8.0|2009-04-13 23:47:15|    0|    0|
+----+--------------+-----+---------+-------------------+-----+-----+
only showing top 5 rows



### fit pipeline model

In [53]:
pipeline_model = pipeline.fit(train_data2)

In [54]:
predictions = pipeline_model.transform(test_data2)

In [55]:
predictions.select('label','prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 20 rows



### evaluation

In [56]:
binary_model = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='prediction')

In [57]:
auc = binary_model.evaluate(predictions, {binary_model.metricName:'areaUnderROC'})
auc

0.7844794832826747

In [58]:
multi_model = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction')

In [59]:
acc = multi_model.evaluate(predictions, {multi_model.metricName:'accuracy'})
acc

0.9077490774907749

In [60]:
# create predictions for the testing data and show confusion matrix
predictions.groupBy('label', 'prediction').count().orderBy('label').show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       1.0|    6|
|    0|       0.0|  218|
|    1|       0.0|   19|
|    1|       1.0|   28|
+-----+----------+-----+



In [61]:
# calculte the elements of the confusion matrix
TN = predictions.filter('prediction = 0 AND label = prediction').count()
TP = predictions.filter('prediction = 1 AND label = prediction').count()
FN = predictions.filter('prediction = 0 AND label != prediction').count()
FP = predictions.filter('prediction = 1 AND label != prediction').count()

In [62]:
# calculate precision and recall
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('precision = {:.2f}\nrecall = {:.2f}'.format(precision, recall))

precision = 0.82
recall = 0.60


In [63]:
acc = (TP+TN)/(TP+TN+FP+FN)
acc

0.9077490774907749

* acc = 89.25% but confusion matrix shows that the recall metrics of "1" class is low => Do resample data

### save pipeline model

In [64]:
# save model
pipeline_model.write().overwrite().save('Pipeline_Customer_Churn')

In [65]:
# load model
pipeline_model2 = PipelineModel.load('Pipeline_Customer_Churn')

In [66]:
# new data
unlabeled_data = test_data2.drop('label')

In [67]:
# new prediction
predictions2 = pipeline_model2.transform(unlabeled_data)

In [68]:
predictions2.show(5)

+----+--------------+-----+---------+-------------------+-----+--------------------+--------------------+--------------------+----------+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|Churn|            features|       rawPrediction|         probability|prediction|
+----+--------------+-----+---------+-------------------+-----+--------------------+--------------------+--------------------+----------+
|22.0|      11254.38| 4.96|      8.0|2012-06-23 08:40:29|    0|[22.0,11254.38,4....|[5.02105753603856...|[0.99344569644221...|       0.0|
|27.0|        8628.8|  5.3|      7.0|2016-04-11 08:58:24|    0|[27.0,8628.8,5.3,...|[5.70883176115783...|[0.99669441831070...|       0.0|
|28.0|       8670.98| 3.99|      6.0|2006-11-05 12:19:57|    0|[28.0,8670.98,3.9...|[7.63269960953802...|[0.99951588295104...|       0.0|
|28.0|      11204.23| 3.67|     11.0|2012-10-25 20:54:44|    0|[28.0,11204.23,3....|[1.99172646216628...|[0.87992566913995...|       0.0|
|29.0|       8688.17|  5.7|      9

# Build model with Resample data

## Oversampling data

In [69]:
train_data2.select('label').groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  103|
|    0|  526|
+-----+-----+



In [70]:
# oversampling "1" class of train_data

one_df = train_data2.filter(train_data2['label']==1)
zero_df = train_data2.filter(train_data2['label']==0)

ratio_one = int(zero_df.count()/one_df.count())

oversample_ratio_one = range(ratio_one)
oversampled_one_df = one_df.withColumn("dummy", 
                                     explode(array([lit(x) for x in oversample_ratio_one]))).drop('dummy')

In [71]:
combined_train_data2 = zero_df.unionAll(oversampled_one_df)

In [72]:
combined_train_data2.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    1|  515|
|    0|  526|
+-----+-----+



## Pipeline with resample data

### fit pipeline model

In [73]:
pipeline_model2 = pipeline.fit(combined_train_data2)

In [74]:
predictions2 = pipeline_model2.transform(test_data2)

In [75]:
predictions2.select('label','prediction').show(10)

+-----+----------+
|label|prediction|
+-----+----------+
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    1|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
|    0|       0.0|
+-----+----------+
only showing top 10 rows



### evaluation

In [76]:
binary_model2 = BinaryClassificationEvaluator(labelCol='label', rawPredictionCol='prediction')

In [77]:
auc2 = binary_model2.evaluate(predictions2, {binary_model2.metricName:'areaUnderROC'})
auc2

0.8305946048632218

In [78]:
multi_model2 = MulticlassClassificationEvaluator(labelCol='label', predictionCol='prediction')

In [79]:
acc2 = multi_model2.evaluate(predictions2, {multi_model2.metricName:'accuracy'})
acc2

0.8450184501845018

In [80]:
# create predictions for the testing data and show confusion matrix
predictions2.groupBy('label', 'prediction').count().orderBy('label').show()

+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|    0|       1.0|   33|
|    0|       0.0|  191|
|    1|       0.0|    9|
|    1|       1.0|   38|
+-----+----------+-----+



In [81]:
# calculte the elements of the confusion matrix
TN = predictions2.filter('prediction = 0 AND label = prediction').count()
TP = predictions2.filter('prediction = 1 AND label = prediction').count()
FN = predictions2.filter('prediction = 0 AND label != prediction').count()
FP = predictions2.filter('prediction = 1 AND label != prediction').count()

In [82]:
# calculate precision and recall
precision = TP/(TP+FP)
recall = TP/(TP+FN)
print('precision = {:.2f}\nrecall = {:.2f}'.format(precision, recall))

precision = 0.54
recall = 0.81


In [83]:
acc2 = (TP+TN)/(TP+TN+FP+FN)
acc2

0.8450184501845018

* acc = 83.87%, recall = 83% and confusion matrix show that this model (with resample data) is better!

### save pipeline model

In [84]:
# save model
pipeline_model2.write().overwrite().save('Pipeline_Customer_Churn_resample')

In [85]:
# load model
pipeline_model2_1 = PipelineModel.load('Pipeline_Customer_Churn_resample')

In [86]:
# new data
unlabeled_data = test_data2.drop('label')

In [87]:
# new prediction
predictions2_1 = pipeline_model2_1.transform(unlabeled_data)

In [88]:
predictions2_1.show(5)

+----+--------------+-----+---------+-------------------+-----+--------------------+--------------------+--------------------+----------+
| Age|Total_Purchase|Years|Num_Sites|       Onboard_date|Churn|            features|       rawPrediction|         probability|prediction|
+----+--------------+-----+---------+-------------------+-----+--------------------+--------------------+--------------------+----------+
|22.0|      11254.38| 4.96|      8.0|2012-06-23 08:40:29|    0|[22.0,11254.38,4....|[3.19216872545509...|[0.96053850658813...|       0.0|
|27.0|        8628.8|  5.3|      7.0|2016-04-11 08:58:24|    0|[27.0,8628.8,5.3,...|[3.80321437248563...|[0.97818741898429...|       0.0|
|28.0|       8670.98| 3.99|      6.0|2006-11-05 12:19:57|    0|[28.0,8670.98,3.9...|[5.67726278928063...|[0.99658876150739...|       0.0|
|28.0|      11204.23| 3.67|     11.0|2012-10-25 20:54:44|    0|[28.0,11204.23,3....|[0.40956412956476...|[0.60098336069106...|       0.0|
|29.0|       8688.17|  5.7|      9

In [89]:
end_time = time.time()

In [90]:
duration = (end_time - start_time)/60
duration

0.8314080834388733